In [1]:
import uol_redacoes_xml
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import mac_morpho
import nltk.corpus
import pandas as pd
import regex as re
import spacy

In [2]:
essays = uol_redacoes_xml.load()

Separar elementos com a mesma função sintática

Antes de conjunções adversativas **x**

Separar orações intercaladas

aposto **x**

Separar expressões explicativas

Isolar o vocativo

Isolar datas

Marcar zeugma

Separar as orações subordinadas adverbiais

Separar as orações subordinadas adverbiais deslocadas

Não separar o sujeito do predicado **x**

Não separar o verbo do complemento **x**

In [3]:
def count_virgula(palavra1,palavra2,frase):
    if palavra1 == None or palavra2  == None or palavra1 == '' or  palavra2 == '':
        return 0
    posicao_gato = frase.find(palavra1)
    posicao_pulou = frase.find(palavra2)

    if posicao_gato != -1 and posicao_pulou != -1:
        substring = frase[posicao_gato:posicao_pulou]
        numero_virgulas = substring.count(',')
        return numero_virgulas
    else:
        return -1

In [4]:
def sujeito_pedicado(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    # Identificar sujeito e predicado
    sujeito = ""
    predicado = ""

    for token in doc:
        if "subj" in token.dep_ and sujeito == "":
            sujeito += token.text
        elif "ROOT" in token.dep_ and predicado == "":
            predicado += token.text

    return {'sujeito': sujeito, 'predicado': predicado}

In [5]:
def error_virgula_suj_pred(frase):
    senter = sujeito_pedicado(frase)

    num = count_virgula(senter['sujeito'],senter['predicado'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [6]:
def identificar_verbo_complemento(frase):
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(frase)
    
    verb = None
    obj = None
    
    for token in doc:
        if "VERB" in token.pos_:
            verb = token.text
            for child in token.children:
                if "obj" in child.dep_:
                    obj = child.text
                    break
    
    return {'verbo': verb, 'obj': obj}


In [7]:
def error_virgula_verbo(frase):
    senter = identificar_verbo_complemento(frase)
    num = count_virgula(senter['verbo'],senter['obj'], frase)
    if num%2 == 0:
        return 0
    elif num == -1:
        return -1
    else:
        return 1

In [8]:
def error_virgula_Conj_adj(frase):
    conjuncoes_adversativas = ['mas', 'porém', 'contudo', 'todavia', 'entretanto', 'no entanto']
    frase = frase.lower() 
    palavras = frase.split()

    for i in range(1, len(palavras)):
        if palavras[i] in conjuncoes_adversativas:
            if palavras[i - 1][-1] == ',':
                return 0
            else:
                return 1

    return -1

In [9]:
def error_virgulas_aposto(frase):
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(frase)

    for token in doc:
        if token.dep_ == 'appos':
            
            rela_appos = [str(child) for child in token.children]
            print(token, rela_appos)
            if rela_appos.count(',') == 2:
                return 0
            else: return 1
    else:
        return -1

In [14]:
def corretor(frase):
    error = 0
    message = ""
    
    e = error_virgula_suj_pred(frase)
    if e == 1:
        error += e
        message += "Comma separating Subject from predicate; "
    
    e = error_virgulas_aposto(frase)
    if e ==1:
        error += e
        message += "Missing comma between the 'aposto'; "

    return error, message

In [11]:
def split_sentences(text):
    regex =  r'(?<=[.!?])\s*(?=(?:[^"]*"[^"]*")*[^"]*$)'  # Expressão regular para separar as sentenças

    sentences = re.split(regex, text)

    return sentences

In [12]:
def get_essays_corrections_tool(essays):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            if sent != "":
                error, message = corretor(sent)
                data.append({
                        'essay': i,
                        'error': error,
                        'message': message,
                        'frase': sent})
    return pd.DataFrame(data)

In [15]:
erros_df = get_essays_corrections_tool(essays)

o ['abre']
liberdade [':', 'expressão']
música [',', 'a']
Michel [',', 'Temer', ',']
Marcela []
" []
Michel []
Federal ['portanto']
caso [',', 'como', 'foi', 'o', 'jornal']
Voltaire []
" []
Paulo ['Freire']
prova [',', 'disso']
Marcela []
Zygmunt ['Bauman']
gerador []
Marcela ['Temer']
" []
Marcela ['Temer']
Monica [',', 'Herman', 'Salem', 'Coggiano', ',']
o ['provoca']
temas [',', 'amor']
violência [':', 'criança', ',']
Júlio ['César']
Pitágoras []
comportamento [',', 'chegam']
o ['acaba']
Jacques ['Rousseau']
isso []
Nicolae ['Iorga']
exemplo ['telhado']
um [',', 'filósofos']
investimento ['nenhum', 'educação']
poder ['força', 'perspicácia']
geradora [',', 'injustiças']
Sérgio ['Holanda']
Fernando ['Collor']
o [',', 'pode', 'advém']
Eduardo [',', 'Cunha']
Paulo ['Freire']
Collor []
o ['acontece', 'o']
acatamento [',', 'leis', ',']
o ['levou']
– ['dinheiro']
CNH []
corrupção ['–']
ação ['(', 'omissão', ')']
combatentes ['-', ',', 'de', 'ordem']
Legislativo ['Executivo']
delação [':', 

In [16]:
erros_df.to_csv('Comma_error.csv', encoding='utf-8')